# Python Implementation of online TSL learning algorithm as presented in [Lambert (2021)](https://proceedings.mlr.press/v153/lambert21a/lambert21a.pdf)

### imports and definitions

In [1]:
from itertools import combinations, product
from tqdm import tqdm

### Helper Methods

In [2]:
def nsorted(collection, key = lambda x:x): 
	'''
	This method implements numerical sorting, rather than default lexicographical sorting of sorted()
	'''
	if collection.__class__ == dict:
		return {key:collection[key] for key in nsorted(collection.keys())}
	return sorted(collection, key = lambda element : (len(key(element)), key(element)))

In [3]:
class Set:
	'''
	This is just a wrapper class around python's set class, which allows Sets to be placed in other Sets
	'''
	def __init__(self, _set = {}):
		self._set = set(_set)
		self._rehash()

	def _rehash(self):
		self._hash = tuple(nsorted(self._set)).__hash__()
	def __hash__(self):
		if self._hash is None:
			self._rehash()
		return self._hash

	def __eq__(self, other):
		return self.__class__ == other.__class__ and self._set == other._set
	def __len__(self):
		return len(self._set)
	def __repr__(self):
		return 'Set({{{0}}})'.format(', '.join(map(repr, nsorted(self._set))))
	def __str__(self):
		return self.__repr__()
	def __lt__(self, other):
		if other.__class__ == self.__class__:
			return self._set.__lt__(other._set)
		return self._set.__lt__(other)
	def __gt__(self, other):
		if other.__class__ == self.__class__:
			return self._set.__gt__(other._set)
		return self._set.__gt__(other)
	def __leq__(self, other):
		if other.__class__ == self.__class__:
			return self._set.__leq__(other._set)
		return self._set.__leq__(other)
	def __geq__(self, other):
		if other.__class__ == self.__class__:
			return self._set.__geq__(other._set)
		return self._set.__geq__(other)
	def __iter__(self):
		return self._set.__iter__()
	def issubset(self, other):
		return self._set.issubset(other._set)
	def issuperset(self, other):
		return self._set.issuperset(other._set)
	def union(self, other):
		return Set((self._set.union(other._set)))
	def difference(self, other):
		return Set(self._set.difference(other._set))

	def add(self, e):
		self._set.add(e)
		self._hash = None; self._ordered = False
	def update(self, es):
		self._set.update(es)
		self._hash = None; self._ordered = False

In [4]:
def dictUnion(a, b):
	'''
	for sets of augmented subsequences encoded as a dict from tuples of symbols to Sets of Sets, this function unions the two sets
	'''
	ans = dict()
	for e in a:
		ans[e] = Set()
	for e in b:
		ans[e] = Set()
	
	for e in a:
		ans[e].update(a[e])
	for e in b:
		ans[e].update(b[e])
	return ans

In [5]:
def width_j_substrings(w, j):
    return tuple(w[i:i+j] for i in range(len(w)-j+1))

In [6]:
class grammar_tuple(tuple):
    '''
    This is just a new definition for __len__ so that the len() of a grammar tuple is meaningful (rather than just always 3)
    '''
    def __len__(self):
        return len(self[1]) + sum(len(item) for item in self[2].values())

### Set the standard symbol and dependency widths

In [7]:
K = 2 # dependency width
M = 2 # symbol width

### Learner definitions/prerequisites

"$ f : \Sigma^* \rightarrow \mathcal{P} \left( \Sigma^{\leq k+1} \right) $
gathers all and only those substrings of $w$ whose width is bounded above by $ k+1 $"

In [8]:
def f(w, k=K):
    return Set  (
                    sum (
                            tuple(width_j_substrings(w, j) for j in range(k+1+1)), #get every j-factor for every value of j up to k+1 inclusive
                            ()
                        )
                )

"$ x : \Sigma^* \rightarrow \mathcal{P} \left( \Sigma^{\leq k} \times \mathcal{P} \left( \Sigma \right) \right) $ extracts the valid augmented subsequences of width bounded above by $k$"

In [9]:
def x(w, k=K):

    symbols_at_indices = lambda indices : tuple(w[index] for index in indices)
    

    augmented_subsequences = dict()         # Create a dictionary from subsequences to the set of their intervener sets
    augmented_subsequences[()] = Set([Set()]) # The only set of symbols that can intervene a length-0 tuple is the empty set

    for j in range(1, k+1):                                                             # iterate across factor lengths j, 1 to k inclusive
        for subsequence_indices in list(combinations(range(len(w)), j)):                # look at each length-j subsequence of indices
            subsequence = symbols_at_indices(subsequence_indices)               # extract the tuple of symbols at those selected indices
            intervening_indices =   [                                                   # compute the intervening indices
                                        intervening_index
                                        for intervening_index in range(subsequence_indices[0], subsequence_indices[-1])
                                        if intervening_index not in subsequence_indices
                                    ]
            intervening_set = Set(symbols_at_indices(intervening_indices))         # extract the set of symbols at the intervening indices

            if set(subsequence).isdisjoint(set(intervening_set)):           # if there are no symbols shared by the subsequence and the interveners, this is a valid augmented subsequence
                if subsequence not in augmented_subsequences:
                    augmented_subsequences[subsequence] = Set()
                augmented_subsequences[subsequence].add(intervening_set)    # add the set of augmented subsequences

    return nsorted(augmented_subsequences)

"$ r :  \mathcal{P} \left( \Sigma^{\leq k} \times \mathcal{P} \left( \Sigma \right) \right) \rightarrow \mathcal{P} \left( \Sigma^{\leq k} \times \mathcal{P} \left( \Sigma \right) \right)$ restricts the set of augmented subsequences to exclude any that are entailed by any other"

In [10]:
def r(augmented_subsequences):
    return  nsorted (
                        {
                            subsequence_symbols: Set((
                                                        intervener_symbol_set
                                                        for intervener_symbol_set in intervener_symbol_sets
                                                        if not any  (
                                                                            intervener_symbol_set.issuperset(other_intervener_symbol_set)
                                                                        and
                                                                            intervener_symbol_set != other_intervener_symbol_set
                                                                        for other_intervener_symbol_set in intervener_symbol_sets
                                                                    )

                                                    ))
                            for subsequence_symbols, intervener_symbol_sets in augmented_subsequences.items()
                        }
                    )

### Learners

"We can define a learner $ \varphi \left( \langle  G_{\ell}, G_s\rangle, w \right) = \langle G_{\ell} \cup f \left( w \right), r \left( G_s \cup x \left( w \right) \right) \rangle$" This is `learn_step`

"The composite grammar can immediately be used as an acceptor without further processing . . . 
$ \mathcal{L} \left( \langle G_{\ell}, G_s \rangle \right) = \{ w : f \left( w \right) \subseteq G_{\ell} \land r \left( G_s \cup x \left( w \right) \right) \subseteq G_s \}$
. In words, a string is accepted iff it has only permitted substrings and each of its valid augmented subsequences is attested or entailed by something that is attested." This is `scan`

In [11]:
class TSL_Learner:
    def __init__(self, k=K):
        self.k = k          # dependency width
        self.G_l = Set()    # substrings of length bounded above by k+1
        self.G_s = dict()   # augmented subsequences of length bounded above by k
        self._data_source = None
    def __repr__(self):
        return f'TSL-{self.k} Grammar\n{self.G_l}\n{self.G_s}'
    def __call__(self, *args, **kwargs):
        return self.scan(*args, **kwargs)
    def extract_alphabet(self):
        pass # This algorithm does not require this step for learning :)

    @property
    def grammar(self):
        return grammar_tuple((self.k, self.G_l, self.G_s))

    #This is an online algorithm, so it does not need a persistent copy of the strings it sees. To highlight this, I have enforced that the learner ONLY streams inputs from an iterator, without retaining a pointer to the complete input 
    @property
    def data(self):
        return (w for w in [])
    @data.setter
    def data(self, W):
        self._data_source = (w for w in W)


    def preprocess(self, w):
        return '>'*(self.k-1) + w + '<'*(self.k-1) # add word-boundary symbols

    def scan(self, w_raw):
        w = self.preprocess(w_raw)
        return  (
                        f(w, k = self.k).issubset(self.G_l)
                    and
                        all (
                                (
                                        subsequence in self.G_s.keys()
                                    and
                                        intervening_sets.issubset(self.G_s[subsequence])
                                )
                                for subsequence, intervening_sets in r(dictUnion(self.G_s, x(w, self.k))).items()
                            )
                )
    
    def learn_step(self, w_raw):
        w = self.preprocess(w_raw)
        self.G_l = self.G_l.union(f(w, k=self.k))
        self.G_s = r(dictUnion(self.G_s, x(w, self.k)))
    
    def learn(self, W=None):
        W = (w for w in (W if W is not None else self._data_source))
        for w in tqdm(W, desc="Learning"):
            self.learn_step(w)

    def generate_sample(self, n, use_iterator=False):
        def generate_with_iterator(n=n):
            alphabet = set(''.join(''.join(substring) for substring in self.G_l)).difference('<','>')

            j = 0
            while True:
                for w in map(''.join, product(alphabet, repeat=j)):
                    if self.scan(w):
                        yield w
                        n -= 1
                        if n == 0:
                            return
                j += 1
        return ((lambda x:x) if use_iterator else list)(tqdm(generate_with_iterator(), total=n))

In [12]:
class ITSL_Learner(TSL_Learner):
    def __init__(self, k=K, m=M):
        super().__init__(k)
        self.m = m             # symbol width
    def __repr__(self):
        return f'ITSL-({self.k}, {self.m}) Grammar\n{self.G_l}\n{self.G_s}'
    @property
    def grammar(self):
        return grammar_tuple(((self.k, self.m), self.G_l, self.G_s))

    def preprocess(self, w):
        return width_j_substrings( #break string into width-m symbols, i.e. symbols created from m adjacent symbols
            '>'*(self.k*self.m-1) + w + '<'*(self.k*self.m-1), # add word-boundary symbols. Adding k*m-1 ensures that the first k-factor of consecutive m-width symbols contains exactly one true symbol, analogous to adding k-1 word boundary symbols for a TSL learner    
            self.m
        )

### Experiments

In [13]:
tsl_args = [TSL_Learner, "tsl", [], {'k':2}]
itsl_args = [ITSL_Learner, "itsl", [], {'k':2, 'm':2}]
from Aksenova import experiments, run_experiment_with_printout

['bbabbaabbb', 'aabbabaaab', 'babbaaabba', 'ooopoooppp', 'ppoopppooo']
['ooxxooxxxx', 'xxxaaxxxxa', 'oxxxxooxxx', 'xxxxooxxxx', 'aaxxxaxxxa']
['ttpppptooo', 'ooppppooot', 'pppaaapppp', 'bboobbobbt', 'ppppaataaa']
Percentage of well-formed words: 100%.
Percentage of well-formed tonal layers: 100%.
['abaaabbbbp', 'bpbbpbapba', 'baappapaaa', 'ppabbbaaap', 'apppbpabbp', 'aabppapapp', 'bpbppbpapp', 'abapbbbpaa', 'abpabpappa', 'paabpabpbp', 'abppbapapp', 'aaabapppaa', 'baabbbapap', 'paapppbbpa', 'aabapapaap']
685618
['Aa\r', 'Aachener\r', 'Aachenerin\r', 'Aachenerinnen\r', 'Aachenern\r', 'Aacheners\r', 'Aaden\r', 'Aak\r', 'Aake\r', 'Aaken\r'] ...
Number of final /b/: 0
Number of final /d/: 0
Number of final /g/: 0
685147
Clean dataset: ['aa\r', 'aachener\r', 'aachenerin\r', 'aachenerinnen\r', 'aachenern\r', 'aacheners\r', 'aaden\r', 'aak\r', 'aake\r', 'aaken\r', 'aakerbeere\r', 'aakerbeeren\r', 'aakes\r', 'aaks\r', 'aal\r'] ...

471
Banned words: ['abbé\r', 'abbés\r', 'abrégé\r', 'abrégés\r'

#### TSL experiments

In [14]:
for experiment in experiments:
    print(experiment['description'])
    run_experiment_with_printout(*tsl_args, *experiment['args'])
    print()

Word-final devoicing, Artificial Grammar


Learning: 1001it [00:00, 2685.50it/s]


Percentage of well-formed words: 100%.
--------------------------
Generates such strings: ['', 'aa', 'ap', 'ba', 'bp', 'pa', 'pp', 'aaa', 'aap', 'aba', 'abp', 'apa', 'app', 'baa', 'bap']
--------------------------
Size of the grammar: 86
--------------------------
Grammars: (2, {'', '<', '>', 'a', 'b', 'p', '><', '>a', '>b', '>p', 'a<', 'aa', 'ab', 'ap', 'ba', 'bb', 'bp', 'p<', 'pa', 'pb', 'pp', '>aa', '>ab', '>ap', '>ba', '>bb', '>bp', '>pa', '>pb', '>pp', 'aa<', 'aaa', 'aab', 'aap', 'aba', 'abb', 'abp', 'ap<', 'apa', 'apb', 'app', 'ba<', 'baa', 'bab', 'bap', 'bba', 'bbb', 'bbp', 'bp<', 'bpa', 'bpb', 'bpp', 'pa<', 'paa', 'pab', 'pap', 'pba', 'pbb', 'pbp', 'pp<', 'ppa', 'ppb', 'ppp'}, {(): {{}}, ('<',): {{}}, ('>',): {{}}, ('a',): {{}}, ('b',): {{}}, ('p',): {{}}, ('>', '<'): {{}}, ('>', 'a'): {{}}, ('>', 'b'): {{}}, ('>', 'p'): {{}}, ('a', '<'): {{}}, ('a', 'a'): {{}}, ('a', 'b'): {{}}, ('a', 'p'): {{}}, ('b', '<'): {{'a'}, {'p'}}, ('b', 'a'): {{}}, ('b', 'b'): {{}}, ('b', 'p'): {{}},

Learning: 685148it [08:02, 1420.20it/s]


Percentage of well-formed words: 100%.
--------------------------
Generates such strings: ['', 'aa', 'ka', 'ta', 'ba', 'pa', 'da', 'ga', 'aaa', 'aka', 'ata', 'aba', 'apa', 'ada', 'aga']
--------------------------
Size of the grammar: 336
--------------------------
Grammars: (2, {'', '<', '>', 'a', 'b', 'd', 'g', 'k', 'p', 't', '><', '>a', '>b', '>d', '>g', '>k', '>p', '>t', 'a<', 'aa', 'ab', 'ad', 'ag', 'ak', 'ap', 'at', 'ba', 'bb', 'bd', 'bg', 'bk', 'bp', 'bt', 'da', 'db', 'dd', 'dg', 'dk', 'dp', 'dt', 'ga', 'gb', 'gd', 'gg', 'gk', 'gp', 'gt', 'ka', 'kb', 'kd', 'kg', 'kk', 'kp', 'kt', 'pa', 'pb', 'pd', 'pg', 'pk', 'pp', 'pt', 'ta', 'tb', 'td', 'tg', 'tk', 'tp', 'tt', '>aa', '>ab', '>ad', '>ag', '>ak', '>ap', '>at', '>ba', '>da', '>ga', '>ka', '>pa', '>pk', '>pt', '>ta', 'aa<', 'aaa', 'aab', 'aad', 'aag', 'aak', 'aap', 'aat', 'aba', 'abb', 'abd', 'abg', 'abk', 'abp', 'abt', 'ada', 'adb', 'add', 'adg', 'adk', 'adp', 'adt', 'aga', 'agb', 'agd', 'agg', 'agk', 'agp', 'agt', 'aka', 'akb', '

...

Percentage of well-formed tonal layers: 23%.
--------------------------
Generates such strings: ['', 'HH', 'HL', 'LH', 'LL', 'HHH', 'HHL', 'HLL', 'LHH', 'LHL', 'LLH', 'LLL', 'HHHH', 'HHHL', 'HHLL']
--------------------------
Size of the grammar: 43
--------------------------
Grammars: (2, {'', '<', '>', 'H', 'L', '><', '>H', '>L', 'H<', 'HH', 'HL', 'L<', 'LH', 'LL', '>HH', '>HL', '>LH', '>LL', 'HH<', 'HHH', 'HHL', 'HL<', 'HLL', 'LH<', 'LHH', 'LHL', 'LL<', 'LLH', 'LLL'}, {(): {{}}, ('<',): {{}}, ('>',): {{}}, ('H',): {{}}, ('L',): {{}}, ('>', '<'): {{}}, ('>', 'H'): {{}}, ('>', 'L'): {{}}, ('H', '<'): {{}}, ('H', 'H'): {{}}, ('H', 'L'): {{}}, ('L', '<'): {{}}, ('L', 'H'): {{}}, ('L', 'L'): {{}}})

First-last harmony, Artificial Grammar


Learning: 1001it [00:00, 2602.36it/s]


Percentage of first-last harmonic words: 49%.
--------------------------
Generates such strings: ['', 'aa', 'ao', 'oa', 'oo', 'aaa', 'aao', 'axa', 'axo', 'aoa', 'aoo', 'oaa', 'oao', 'oxa', 'oxo']
--------------------------
Size of the grammar: 83
--------------------------
Grammars: (2, {'', '<', '>', 'a', 'o', 'x', '><', '>a', '>o', 'a<', 'aa', 'ao', 'ax', 'o<', 'oa', 'oo', 'ox', 'xa', 'xo', 'xx', '>aa', '>ao', '>ax', '>oa', '>oo', '>ox', 'aa<', 'aaa', 'aao', 'aax', 'ao<', 'aoa', 'aoo', 'aox', 'axa', 'axo', 'axx', 'oa<', 'oaa', 'oao', 'oax', 'oo<', 'ooa', 'ooo', 'oox', 'oxa', 'oxo', 'oxx', 'xa<', 'xaa', 'xao', 'xax', 'xo<', 'xoa', 'xoo', 'xox', 'xxa', 'xxo', 'xxx'}, {(): {{}}, ('<',): {{}}, ('>',): {{}}, ('a',): {{}}, ('o',): {{}}, ('x',): {{}}, ('>', '<'): {{}}, ('>', 'a'): {{}}, ('>', 'o'): {{}}, ('>', 'x'): {{'o'}, {'a'}}, ('a', '<'): {{}}, ('a', 'a'): {{}}, ('a', 'o'): {{}}, ('a', 'x'): {{}}, ('o', '<'): {{}}, ('o', 'a'): {{}}, ('o', 'o'): {{}}, ('o', 'x'): {{}}, ('x', '<'): {{'o'

Learning: 997it [00:00, 1610.59it/s]


Percentage of harmonic words: 100%.
set()
--------------------------
Generates such strings: ['', 'aa', 'ax', 'ab', 'ao', 'ap', 'ay', 'ad', 'ae', 'xa', 'xx', 'xb', 'xo', 'xp', 'xy']
--------------------------
Size of the grammar: 804
--------------------------
Grammars: (2, {'', '<', '>', 'a', 'b', 'd', 'e', 'o', 'p', 'x', 'y', '><', '>a', '>b', '>d', '>e', '>o', '>p', '>x', '>y', 'a<', 'aa', 'ab', 'ad', 'ae', 'ao', 'ap', 'ax', 'ay', 'b<', 'ba', 'bb', 'bd', 'be', 'bo', 'bp', 'bx', 'by', 'd<', 'da', 'db', 'dd', 'de', 'do', 'dp', 'dx', 'dy', 'e<', 'ea', 'eb', 'ed', 'ee', 'eo', 'ep', 'ex', 'ey', 'o<', 'oa', 'ob', 'od', 'oe', 'oo', 'op', 'ox', 'oy', 'p<', 'pa', 'pb', 'pd', 'pe', 'po', 'pp', 'px', 'py', 'x<', 'xa', 'xb', 'xd', 'xe', 'xo', 'xp', 'xx', 'xy', 'y<', 'ya', 'yb', 'yd', 'ye', 'yo', 'yp', 'yx', 'yy', '>aa', '>ab', '>ad', '>ae', '>ao', '>ap', '>ax', '>ay', '>ba', '>bb', '>bd', '>be', '>bo', '>bp', '>bx', '>by', '>da', '>db', '>dd', '>de', '>do', '>dp', '>dx', '>dy', '>ea', '>eb', '>

#### ITSL experiments

In [15]:
for experiment in experiments:
    print(experiment['description'])
    run_experiment_with_printout(*itsl_args, *experiment['args'])
    print()

Word-final devoicing, Artificial Grammar


Learning: 1001it [00:04, 232.70it/s]


Percentage of well-formed words: 100%.
--------------------------
Generates such strings: ['', 'aapa', 'aapp', 'aaba', 'apaa', 'abaa', 'ppbp', 'pbpp', 'pbbp', 'aaapa', 'aaapp', 'aaaba', 'aapaa', 'aapap', 'aappa']
--------------------------
Size of the grammar: 927
--------------------------
Grammars: ((2, 2), {(), ('<<',), ('><',), ('>>',), ('>a',), ('>b',), ('>p',), ('a<',), ('aa',), ('ab',), ('ap',), ('ba',), ('bb',), ('bp',), ('p<',), ('pa',), ('pb',), ('pp',), ('<<', '<<'), ('><', '<<'), ('>>', '><'), ('>>', '>>'), ('>>', '>a'), ('>>', '>b'), ('>>', '>p'), ('>a', 'aa'), ('>a', 'ab'), ('>a', 'ap'), ('>b', 'ba'), ('>b', 'bb'), ('>b', 'bp'), ('>p', 'pa'), ('>p', 'pb'), ('>p', 'pp'), ('a<', '<<'), ('aa', 'a<'), ('aa', 'aa'), ('aa', 'ab'), ('aa', 'ap'), ('ab', 'ba'), ('ab', 'bb'), ('ab', 'bp'), ('ap', 'p<'), ('ap', 'pa'), ('ap', 'pb'), ('ap', 'pp'), ('ba', 'a<'), ('ba', 'aa'), ('ba', 'ab'), ('ba', 'ap'), ('bb', 'ba'), ('bb', 'bb'), ('bb', 'bp'), ('bp', 'p<'), ('bp', 'pa'), ('bp', 'pb'),

Learning: 685148it [4:02:33, 47.08it/s]


Percentage of well-formed words: 100%.
--------------------------
Generates such strings: ['', 'aaa', 'ata', 'aba', 'ada', 'kaa', 'taa', 'baa', 'paa', 'daa', 'gaa', 'aaaa', 'aaka', 'aata', 'aaba']
--------------------------
Size of the grammar: 6193
--------------------------
Grammars: ((2, 2), {(), ('<<',), ('><',), ('>>',), ('>a',), ('>b',), ('>d',), ('>g',), ('>k',), ('>p',), ('>t',), ('a<',), ('aa',), ('ab',), ('ad',), ('ag',), ('ak',), ('ap',), ('at',), ('ba',), ('bb',), ('bd',), ('bg',), ('bk',), ('bp',), ('bt',), ('da',), ('db',), ('dd',), ('dg',), ('dk',), ('dp',), ('dt',), ('ga',), ('gb',), ('gd',), ('gg',), ('gk',), ('gp',), ('gt',), ('ka',), ('kb',), ('kd',), ('kg',), ('kk',), ('kp',), ('kt',), ('pa',), ('pb',), ('pd',), ('pg',), ('pk',), ('pp',), ('pt',), ('ta',), ('tb',), ('td',), ('tg',), ('tk',), ('tp',), ('tt',), ('<<', '<<'), ('><', '<<'), ('>>', '><'), ('>>', '>>'), ('>>', '>a'), ('>>', '>b'), ('>>', '>d'), ('>>', '>g'), ('>>', '>k'), ('>>', '>p'), ('>>', '>t'), ('>a'

...

Percentage of well-formed tonal layers: 100%.
--------------------------
Generates such strings: ['', 'HHH', 'HHL', 'HLL', 'LHH', 'LLH', 'LLL', 'HHHH', 'HHHL', 'HHLL', 'HLLL', 'LHHH', 'LHHL', 'LHLL', 'LLHH']
--------------------------
Size of the grammar: 163
--------------------------
Grammars: ((2, 2), {(), ('<<',), ('><',), ('>>',), ('>H',), ('>L',), ('H<',), ('HH',), ('HL',), ('L<',), ('LH',), ('LL',), ('<<', '<<'), ('><', '<<'), ('>>', '><'), ('>>', '>>'), ('>>', '>H'), ('>>', '>L'), ('>H', 'HH'), ('>H', 'HL'), ('>L', 'LH'), ('>L', 'LL'), ('H<', '<<'), ('HH', 'H<'), ('HH', 'HH'), ('HH', 'HL'), ('HL', 'L<'), ('HL', 'LL'), ('L<', '<<'), ('LH', 'H<'), ('LH', 'HH'), ('LH', 'HL'), ('LL', 'L<'), ('LL', 'LH'), ('LL', 'LL'), ('><', '<<', '<<'), ('>>', '><', '<<'), ('>>', '>>', '><'), ('>>', '>>', '>H'), ('>>', '>>', '>L'), ('>>', '>H', 'HH'), ('>>', '>H', 'HL'), ('>>', '>L', 'LH'), ('>>', '>L', 'LL'), ('>H', 'HH', 'HH'), ('>H', 'HH', 'HL'), ('>H', 'HL', 'LL'), ('>L', 'LH', 'HH'), ('>L', '

Learning: 1001it [00:02, 430.70it/s]


Percentage of first-last harmonic words: 100%.
--------------------------
Generates such strings: ['', 'aaxa', 'axaa', 'axxa', 'aooa', 'oaoo', 'oxxo', 'oxoo', 'ooao', 'ooxo', 'aaaxa', 'aaxaa', 'aaxxa', 'aaxoa', 'aaoxa']
--------------------------
Size of the grammar: 625
--------------------------
Grammars: ((2, 2), {(), ('<<',), ('><',), ('>>',), ('>a',), ('>o',), ('a<',), ('aa',), ('ao',), ('ax',), ('o<',), ('oa',), ('oo',), ('ox',), ('xa',), ('xo',), ('xx',), ('<<', '<<'), ('><', '<<'), ('>>', '><'), ('>>', '>>'), ('>>', '>a'), ('>>', '>o'), ('>a', 'aa'), ('>a', 'ao'), ('>a', 'ax'), ('>o', 'oa'), ('>o', 'oo'), ('>o', 'ox'), ('a<', '<<'), ('aa', 'a<'), ('aa', 'aa'), ('aa', 'ao'), ('aa', 'ax'), ('ao', 'o<'), ('ao', 'oa'), ('ao', 'oo'), ('ao', 'ox'), ('ax', 'xa'), ('ax', 'xo'), ('ax', 'xx'), ('o<', '<<'), ('oa', 'a<'), ('oa', 'aa'), ('oa', 'ao'), ('oa', 'ax'), ('oo', 'o<'), ('oo', 'oa'), ('oo', 'oo'), ('oo', 'ox'), ('ox', 'xa'), ('ox', 'xo'), ('ox', 'xx'), ('xa', 'a<'), ('xa', 'aa'), (

Learning: 997it [00:40, 24.34it/s] 


Percentage of harmonic words: 100%.
set()
--------------------------
Generates such strings: ['', 'ab', 'ap', 'ay', 'ad', 'xb', 'xp', 'xy', 'xd', 'ba', 'bx', 'bo', 'be', 'pa', 'px']
--------------------------
Size of the grammar: 14353
--------------------------
Grammars: ((2, 2), {(), ('<<',), ('><',), ('>>',), ('>a',), ('>b',), ('>d',), ('>e',), ('>o',), ('>p',), ('>x',), ('>y',), ('a<',), ('aa',), ('ab',), ('ad',), ('ae',), ('ao',), ('ap',), ('ax',), ('ay',), ('b<',), ('ba',), ('bb',), ('bd',), ('be',), ('bo',), ('bp',), ('bx',), ('by',), ('d<',), ('da',), ('db',), ('dd',), ('de',), ('do',), ('dp',), ('dx',), ('dy',), ('e<',), ('ea',), ('eb',), ('ed',), ('ee',), ('eo',), ('ep',), ('ex',), ('ey',), ('o<',), ('oa',), ('ob',), ('od',), ('oe',), ('oo',), ('op',), ('ox',), ('oy',), ('p<',), ('pa',), ('pb',), ('pd',), ('pe',), ('po',), ('pp',), ('px',), ('py',), ('x<',), ('xa',), ('xb',), ('xd',), ('xe',), ('xo',), ('xp',), ('xx',), ('xy',), ('y<',), ('ya',), ('yb',), ('yd',), ('ye',), ('